In [86]:
import csv, json
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

df = pd.read_csv('/Users/jilljenn/code/qna/data/factor-analysis-data.csv')
qmatrix = pd.read_csv('/Users/jilljenn/code/qna/data/q-info.csv')

In [87]:
df.head()

,stu_id,q_txt_id,q_seen_timestamp,answer_timestamp,is_correct
0,1011,1076,1472651060,1472651117,1
1,1011,1077,1472651117,1472651126,0
2,1011,1077,1472651126,1472651138,1
3,1011,1078,1472651138,1472651144,1
4,1011,1079,1472651144,1472651149,1


In [88]:
students = df['stu_id'].unique()
questions = pd.concat((df['q_txt_id'], qmatrix['q_txt_id'])).unique()
USER_NUM = len(students)
ITEM_NUM = len(questions)

encode_stu = dict(zip(students, range(USER_NUM)))
encode_q = dict(zip(questions, range(ITEM_NUM)))

In [89]:
df['user_id'] = df['stu_id'].map(encode_stu)
df['item_id'] = df['q_txt_id'].map(encode_q)

In [90]:
df.head()

,stu_id,q_txt_id,q_seen_timestamp,answer_timestamp,is_correct,user_id,item_id
0,1011,1076,1472651060,1472651117,1,0,0
1,1011,1077,1472651117,1472651126,0,0,1
2,1011,1077,1472651126,1472651138,1,0,1
3,1011,1078,1472651138,1472651144,1,0,2
4,1011,1079,1472651144,1472651149,1,0,3


In [91]:
len(questions)

246

# QMatrix

In [92]:
qmatrix.head()

,qset_id,q_txt_id
0,3,1070
1,3,1071
2,3,1072
3,3,1073
4,3,1074


In [93]:
skills = qmatrix['qset_id'].unique()
SKILL_NUM = len(skills)
encode_skill = dict(zip(skills, range(SKILL_NUM)))

qmatrix['item_id'] = qmatrix['q_txt_id'].map(encode_q)
qmatrix['skill_id'] = qmatrix['qset_id'].map(encode_skill)

In [100]:
qmatrix.head()

,qset_id,q_txt_id,item_id,skill_id
0,3,1070,14,0
1,3,1071,15,0
2,3,1072,16,0
3,3,1073,17,0
4,3,1074,18,0


In [104]:
rows = qmatrix['item_id']
cols = qmatrix['skill_id']
sp_qmatrix = coo_matrix(([1] * ITEM_NUM, (rows, cols)), shape=(ITEM_NUM, SKILL_NUM)).tocsr()

In [113]:
from scipy.sparse import save_npz
import os.path

DATA_DIR = '/Users/jilljenn/code/TF-recomm/data/berkeley/'
save_npz(os.path.join(DATA_DIR, 'qmatrix.npz'), sp_qmatrix)

# Number of attempts

In [107]:
from collections import Counter

acc_wins = Counter()
acc_fails = Counter()
nb_wins = []
nb_fails = []
for user_id, work_id, outcome in np.array(df[['user_id', 'item_id', 'is_correct']]):
    nb_wins.append(acc_wins[user_id, work_id])
    nb_fails.append(acc_fails[user_id, work_id])
    if outcome == 1:
        acc_wins[user_id, work_id] += 1
    else:
        acc_fails[user_id, work_id] += 1

In [108]:
df['nb_wins'] = nb_wins
df['nb_fails'] = nb_fails

In [109]:
df.head()

,stu_id,q_txt_id,q_seen_timestamp,answer_timestamp,is_correct,user_id,item_id,nb_wins,nb_fails
0,1011,1076,1472651060,1472651117,1,0,0,0,0
1,1011,1077,1472651117,1472651126,0,0,1,0,0
2,1011,1077,1472651126,1472651138,1,0,1,0,1
3,1011,1078,1472651138,1472651144,1,0,2,0,0
4,1011,1079,1472651144,1472651149,1,0,3,0,0


# Cross-validation

In [111]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df[['user_id', 'item_id', 'is_correct', 'nb_wins', 'nb_fails']], test_size=0.2)

In [112]:
len(train), len(test)

(449760, 112441)

In [114]:
train.to_csv(os.path.join(DATA_DIR, 'train.csv'), header=False, index=False)
test.to_csv(os.path.join(DATA_DIR, 'val.csv'), header=False, index=False)
test.to_csv(os.path.join(DATA_DIR, 'test.csv'), header=False, index=False)

In [115]:
import yaml

with open(os.path.join(DATA_DIR, 'config.yml'), 'w') as f:
    config = {
        'USER_NUM': USER_NUM,
        'ITEM_NUM': ITEM_NUM,
        'NB_CLASSES': 2,
        'BATCH_SIZE': len(train)
    }
    f.write(yaml.dump(config, default_flow_style=False))